In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession # SparkSession 是Spark 2.0版本的新入口
spark = SparkSession.builder.master('local').getOrCreate()

In [13]:
data = []
for i in range(9):
    for j in range(i+1):
        data.append((i,j))
data

[(0, 0),
 (1, 0),
 (1, 1),
 (2, 0),
 (2, 1),
 (2, 2),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8)]

In [14]:
rdd = spark.createDataFrame(data).rdd

In [15]:
print(rdd.collect())

[Row(_1=0, _2=0), Row(_1=1, _2=0), Row(_1=1, _2=1), Row(_1=2, _2=0), Row(_1=2, _2=1), Row(_1=2, _2=2), Row(_1=3, _2=0), Row(_1=3, _2=1), Row(_1=3, _2=2), Row(_1=3, _2=3), Row(_1=4, _2=0), Row(_1=4, _2=1), Row(_1=4, _2=2), Row(_1=4, _2=3), Row(_1=4, _2=4), Row(_1=5, _2=0), Row(_1=5, _2=1), Row(_1=5, _2=2), Row(_1=5, _2=3), Row(_1=5, _2=4), Row(_1=5, _2=5), Row(_1=6, _2=0), Row(_1=6, _2=1), Row(_1=6, _2=2), Row(_1=6, _2=3), Row(_1=6, _2=4), Row(_1=6, _2=5), Row(_1=6, _2=6), Row(_1=7, _2=0), Row(_1=7, _2=1), Row(_1=7, _2=2), Row(_1=7, _2=3), Row(_1=7, _2=4), Row(_1=7, _2=5), Row(_1=7, _2=6), Row(_1=7, _2=7), Row(_1=8, _2=0), Row(_1=8, _2=1), Row(_1=8, _2=2), Row(_1=8, _2=3), Row(_1=8, _2=4), Row(_1=8, _2=5), Row(_1=8, _2=6), Row(_1=8, _2=7), Row(_1=8, _2=8)]


In [16]:
pair_rdd = rdd.map(lambda x:(x[0]+1,[x[1]]))

In [17]:
pair_rdd.collect()

[(1, [0]),
 (2, [0]),
 (2, [1]),
 (3, [0]),
 (3, [1]),
 (3, [2]),
 (4, [0]),
 (4, [1]),
 (4, [2]),
 (4, [3]),
 (5, [0]),
 (5, [1]),
 (5, [2]),
 (5, [3]),
 (5, [4]),
 (6, [0]),
 (6, [1]),
 (6, [2]),
 (6, [3]),
 (6, [4]),
 (6, [5]),
 (7, [0]),
 (7, [1]),
 (7, [2]),
 (7, [3]),
 (7, [4]),
 (7, [5]),
 (7, [6]),
 (8, [0]),
 (8, [1]),
 (8, [2]),
 (8, [3]),
 (8, [4]),
 (8, [5]),
 (8, [6]),
 (8, [7]),
 (9, [0]),
 (9, [1]),
 (9, [2]),
 (9, [3]),
 (9, [4]),
 (9, [5]),
 (9, [6]),
 (9, [7]),
 (9, [8])]

In [30]:
n = 1
def get_list(x,y):
    global n
    n+=1
    if n%2 is 1:
        return x+y
    return x

In [53]:
pair_rdd_result = pair_rdd.reduceByKey(get_list)
pair_rdd_result.collect()

[(1, [0]),
 (2, [0]),
 (3, [0, 1]),
 (4, [0, 1, 3]),
 (5, [0, 2, 4]),
 (6, [0, 2, 4]),
 (7, [0, 1, 3, 5]),
 (8, [0, 1, 3, 5, 7]),
 (9, [0, 2, 4, 6, 8])]

In [42]:
pair_rdd_flat = pair_rdd_result.flatMapValues(lambda x:x[:])

In [44]:
pair_rdd_flat.collect()

[(1, 0),
 (2, 0),
 (3, 0),
 (3, 1),
 (4, 0),
 (4, 1),
 (4, 3),
 (5, 0),
 (5, 2),
 (5, 4),
 (6, 0),
 (6, 2),
 (6, 4),
 (7, 0),
 (7, 1),
 (7, 3),
 (7, 5),
 (8, 0),
 (8, 1),
 (8, 3),
 (8, 5),
 (8, 7),
 (9, 0),
 (9, 2),
 (9, 4),
 (9, 6),
 (9, 8)]

# 日期处理

In [100]:
from dateutil.parser import  parse


In [112]:
t=parse("March 31,2019")
t

datetime.datetime(2019, 3, 31, 0, 0)

In [125]:
try:
    parse("2019-02-29")
    print('ok')
except ValueError:
    print('error')

error


In [72]:
parse("2018/03/21")

datetime.datetime(2018, 3, 21, 0, 0)

In [86]:
time.time("%Y-%m-%d",t.time)

TypeError: time() takes no arguments (2 given)

In [101]:
str(t.date())

'2019-03-21'

In [110]:
str(parse("March 21,2019").date())

'2019-03-21'

In [104]:
s = '℃'

In [108]:
'C' in s

False

In [122]:
import dateutil.parser as par

In [ ]:
par.